In [6]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import json
from pathlib import Path
import time
import re
from datetime import datetime

class USCISDataExtractor:
    """Extract H1B and I-140 data from USCIS quarterly reports"""
    
    def __init__(self, fiscal_year=2025, quarter=3, output_dir='./uscis_output'):
        self.base_url = "https://www.uscis.gov"
        self.index_url = "https://www.uscis.gov/tools/reports-and-studies/immigration-and-citizenship-data"
        self.fiscal_year = fiscal_year
        self.quarter = quarter
        
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        
        # Set up output directory with FY and Quarter
        base_output = Path(output_dir)
        self.output_dir = base_output / f"FY{fiscal_year}_Q{quarter}"
        try:
            self.output_dir.mkdir(parents=True, exist_ok=True)
            print(f"✓ Output directory: {self.output_dir.absolute()}\n")
        except Exception as e:
            print(f"Warning: Using /tmp directory due to: {e}")
            self.output_dir = Path(f'/tmp/uscis_output/FY{fiscal_year}_Q{quarter}')
            self.output_dir.mkdir(parents=True, exist_ok=True)
        
        # Target file patterns (works across all quarters)
        self.target_patterns = {
            'all_forms': [
                'all uscis application and petition form types',
                'all application and petition form types',
                'all form types'
            ],
            'i140_quarter': [
                'i-140 by fiscal year, quarter and case status',
                'i140 by fiscal year quarter',
                'form i-140 by fiscal year'
            ],
            'i140_preference': [
                'i-140, receipts and current status by preference and country',
                'i-140 receipts and current status by preference',
                'i140 preference country'
            ],
            'eb_awaiting': [
                'approved eb petitions awaiting visa',
                'eb petitions awaiting visa',
                'i-140, i-360, i-526 approved eb'
            ]
        }
    
    def fetch_page(self, url):
        """Fetch and parse a webpage"""
        try:
            print(f"Fetching: {url}")
            response = self.session.get(url, timeout=30)
            response.raise_for_status()
            print(f"✓ Status: {response.status_code}\n")
            return BeautifulSoup(response.content, 'html.parser')
        except Exception as e:
            print(f"✗ Error: {e}\n")
            return None
    
    def find_quarterly_page(self, soup):
        """Find the quarterly reports page for specified FY and Quarter"""
        print(f"Searching for FY{self.fiscal_year} Q{self.quarter} page...")
        
        # Search patterns for the target quarter
        patterns = [
            rf'fiscal\s+year\s+{self.fiscal_year}.*quarter\s+{self.quarter}',
            rf'fy\s*{self.fiscal_year}.*q\s*{self.quarter}',
            rf'{self.fiscal_year}.*q{self.quarter}',
        ]
        
        links = soup.find_all('a', href=True)
        candidates = []
        
        for link in links:
            text = link.get_text(strip=True)
            href = link['href']
            
            # Check if link matches any pattern
            for pattern in patterns:
                if re.search(pattern, text, re.IGNORECASE):
                    full_url = urljoin(self.base_url, href)
                    candidates.append({
                        'text': text,
                        'url': full_url,
                        'relevance': self._calculate_relevance(text)
                    })
                    print(f"  → Found candidate: {text[:70]}...")
                    break
        
        if not candidates:
            print(f"⚠ Could not find FY{self.fiscal_year} Q{self.quarter} page")
            print("Attempting to construct URL...")
            # Try to construct the URL based on common patterns
            constructed_url = self._construct_quarterly_url()
            return constructed_url
        
        # Sort by relevance and return best match
        candidates.sort(key=lambda x: x['relevance'], reverse=True)
        best_match = candidates[0]
        print(f"✓ Selected: {best_match['text']}\n")
        return best_match['url']
    
    def _calculate_relevance(self, text):
        """Calculate relevance score for quarterly page link"""
        score = 0
        text_lower = text.lower()
        
        # Exact matches get higher scores
        if f'quarter {self.quarter}' in text_lower or f'q{self.quarter}' in text_lower:
            score += 10
        if str(self.fiscal_year) in text:
            score += 10
        if 'data report' in text_lower:
            score += 5
        if 'quarterly' in text_lower:
            score += 3
        
        return score
    
    def _construct_quarterly_url(self):
        """Construct quarterly URL based on common USCIS patterns"""
        # Common URL patterns used by USCIS
        patterns = [
            f"{self.base_url}/tools/reports-and-studies/immigration-and-citizenship-data/data-set/all-uscis-application-and-petition-form-types-fy{self.fiscal_year}-q{self.quarter}",
            f"{self.base_url}/tools/reports-and-studies/immigration-and-citizenship-data/quarterly-reports-fy{self.fiscal_year}-q{self.quarter}",
        ]
        
        print("Trying constructed URLs:")
        for url in patterns:
            print(f"  Testing: {url}")
            try:
                response = self.session.head(url, timeout=10)
                if response.status_code == 200:
                    print(f"  ✓ Found valid URL\n")
                    return url
            except:
                continue
        
        print("  ⚠ Could not find valid URL\n")
        return patterns[0]  # Return first pattern as fallback
    
    def find_target_files(self, soup, page_url):
        """Find the 4 specific Excel files using flexible pattern matching"""
        print("=" * 70)
        print("SEARCHING FOR TARGET FILES")
        print("=" * 70)
        
        found_files = {}
        links = soup.find_all('a', href=True)
        
        for link in links:
            href = link['href']
            text = link.get_text(strip=True)
            
            # Check if it's an Excel file
            if not (href.lower().endswith('.xlsx') or href.lower().endswith('.xls')):
                continue
            
            text_lower = text.lower()
            
            # Check against each target file pattern
            for key, patterns in self.target_patterns.items():
                if key in found_files:
                    continue  # Already found this file
                
                for pattern in patterns:
                    if pattern in text_lower:
                        # Additional check: make sure it matches the fiscal year/quarter
                        if self._matches_quarter(text):
                            full_url = urljoin(page_url, href)
                            filename = Path(href).name
                            
                            found_files[key] = {
                                'title': text,
                                'url': full_url,
                                'filename': filename
                            }
                            print(f"✓ Found: {key}")
                            print(f"  Title: {text[:60]}...")
                            print(f"  File: {filename}\n")
                            break
        
        # Check if we found all files
        missing = set(self.target_patterns.keys()) - set(found_files.keys())
        if missing:
            print(f"⚠ Missing files: {', '.join(missing)}")
            print("These files may not be available for this quarter yet.\n")
        
        return found_files
    
    def _matches_quarter(self, text):
        """Check if file title matches the target fiscal year and quarter"""
        text_lower = text.lower()
        
        # Check for FY and Quarter
        fy_patterns = [f'fy{self.fiscal_year}', f'fiscal year {self.fiscal_year}', f'{self.fiscal_year}']
        q_patterns = [f'q{self.quarter}', f'quarter {self.quarter}', f'q {self.quarter}']
        
        has_fy = any(pattern in text_lower for pattern in fy_patterns)
        has_q = any(pattern in text_lower for pattern in q_patterns)
        
        # If both FY and Q are mentioned, they must match
        # If only one is mentioned, it should still match
        return has_fy and has_q
    
    def download_file(self, url, filename):
        """Download a file from URL"""
        try:
            print(f"Downloading: {filename}...")
            response = self.session.get(url, timeout=60)
            response.raise_for_status()
            
            downloads_dir = self.output_dir / 'downloads'
            downloads_dir.mkdir(exist_ok=True)
            filepath = downloads_dir / filename
            
            with open(filepath, 'wb') as f:
                f.write(response.content)
            
            size_kb = len(response.content) / 1024
            print(f"✓ Downloaded: {filename} ({size_kb:.1f} KB)\n")
            return filepath
        except Exception as e:
            print(f"✗ Error downloading {filename}: {e}\n")
            return None
    
    def extract_rows_16_17(self, filepath):
        """Extract rows 16 & 17 from All Forms file (I-129 and I-140 data)"""
        print("=" * 70)
        print(f"EXTRACTING ROWS 16-17 FROM: {filepath.name}")
        print("=" * 70)
        
        try:
            # Read Excel file
            xls = pd.ExcelFile(filepath)
            print(f"Available sheets: {xls.sheet_names}\n")
            
            df = pd.read_excel(filepath, sheet_name=0)
            print(f"Total rows: {len(df)}")
            print(f"Total columns: {len(df.columns)}")
            print(f"Columns: {list(df.columns)}\n")
            
            # Extract rows 16 & 17 (0-indexed: 15 and 16)
            if len(df) >= 17:
                rows_16_17 = df.iloc[15:17]
                
                print("ROWS 16-17 DATA:")
                print("-" * 70)
                print(rows_16_17.to_string(index=True))
                print("-" * 70)
                
                # Save to CSV
                output_file = self.output_dir / f"rows_16_17_I129_I140.csv"
                rows_16_17.to_csv(output_file, index=True)
                print(f"\n✓ Saved to: {output_file}\n")
                
                # Display form types
                if len(df.columns) > 0:
                    form_col = df.columns[0]
                    print("\nFORM TYPE DETAILS:")
                    print("-" * 70)
                    for idx in [15, 16]:
                        if idx < len(df):
                            form_type = df.iloc[idx][form_col]
                            print(f"Row {idx+1}: {form_type}")
                    print()
                
                return rows_16_17
            else:
                print(f"⚠ File has only {len(df)} rows (need at least 17)\n")
                return None
                
        except Exception as e:
            print(f"✗ Error: {e}\n")
            return None
    
    def analyze_all_forms(self, filepath):
        """Search for I-129 and I-140 data in All Forms file"""
        print("=" * 70)
        print("ANALYZING I-129 AND I-140 DATA")
        print("=" * 70)
        
        try:
            df = pd.read_excel(filepath, sheet_name=0)
            form_col = df.columns[0]
            
            # Find I-129 rows
            i129_mask = df[form_col].astype(str).str.contains('I-129', case=False, na=False)
            i129_data = df[i129_mask]
            
            # Find I-140 rows
            i140_mask = df[form_col].astype(str).str.contains('I-140', case=False, na=False)
            i140_data = df[i140_mask]
            
            results = {}
            
            # Display I-129 data
            if not i129_data.empty:
                print("\nI-129 (H-1B & NONIMMIGRANT WORKERS):")
                print("-" * 70)
                print(i129_data.to_string(index=True))
                print()
                results['I-129'] = i129_data.to_dict('records')
                
                # Save separately
                i129_file = self.output_dir / 'I129_data.csv'
                i129_data.to_csv(i129_file, index=True)
                print(f"✓ Saved I-129 data to: {i129_file}\n")
            
            # Display I-140 data
            if not i140_data.empty:
                print("I-140 (IMMIGRANT WORKERS):")
                print("-" * 70)
                print(i140_data.to_string(index=True))
                print()
                results['I-140'] = i140_data.to_dict('records')
                
                # Save separately
                i140_file = self.output_dir / 'I140_data.csv'
                i140_data.to_csv(i140_file, index=True)
                print(f"✓ Saved I-140 data to: {i140_file}\n")
            
            # Save JSON summary
            if results:
                json_file = self.output_dir / 'I129_I140_summary.json'
                with open(json_file, 'w') as f:
                    json.dump(results, f, indent=2, default=str)
                print(f"✓ Saved JSON summary to: {json_file}\n")
            
            return results
            
        except Exception as e:
            print(f"✗ Error: {e}\n")
            return None
    
    def process_i140_files(self, downloaded_files):
        """Process the three I-140 specific files"""
        print("=" * 70)
        print("PROCESSING I-140 DETAILED REPORTS")
        print("=" * 70)
        
        i140_keys = ['i140_quarter', 'i140_preference', 'eb_awaiting']
        
        for key in i140_keys:
            if key in downloaded_files:
                filepath = downloaded_files[key]
                print(f"\nProcessing: {filepath.name}")
                print("-" * 70)
                
                try:
                    df = pd.read_excel(filepath, sheet_name=0)
                    print(f"Rows: {len(df)}, Columns: {len(df.columns)}")
                    print("\nFirst 10 rows:")
                    print(df.head(10).to_string())
                    print()
                    
                    # Save as CSV for easier viewing
                    csv_file = self.output_dir / f"{filepath.stem}.csv"
                    df.to_csv(csv_file, index=False)
                    print(f"✓ Saved to: {csv_file}\n")
                    
                except Exception as e:
                    print(f"✗ Error: {e}\n")
    
    def run_extraction(self):
        """Main extraction workflow"""
        print("=" * 70)
        print("USCIS H1B DATA EXTRACTOR")
        print("=" * 70)
        print(f"Target: Fiscal Year {self.fiscal_year}, Quarter {self.quarter}")
        print("=" * 70)
        print()
        
        # Step 1: Fetch the index page
        print("[Step 1/5] Fetching USCIS data index page...")
        soup = self.fetch_page(self.index_url)
        
        if not soup:
            print("✗ Failed to fetch index page. Exiting.\n")
            return
        
        # Step 2: Find quarterly page
        print(f"[Step 2/5] Finding FY{self.fiscal_year} Q{self.quarter} page...")
        quarterly_url = self.find_quarterly_page(soup)
        
        if not quarterly_url:
            print("✗ Could not locate quarterly page. Exiting.\n")
            return
        
        print(f"Using URL: {quarterly_url}\n")
        
        # Step 3: Fetch quarterly page and find files
        print("[Step 3/5] Fetching quarterly page and searching for files...")
        quarterly_soup = self.fetch_page(quarterly_url)
        
        if not quarterly_soup:
            print("✗ Failed to fetch quarterly page. Exiting.\n")
            return
        
        found_files = self.find_target_files(quarterly_soup, quarterly_url)
        
        if not found_files:
            print("✗ No target files found. Exiting.\n")
            return
        
        print(f"✓ Found {len(found_files)}/4 target files\n")
        
        # Step 4: Download files
        print("[Step 4/5] Downloading files...")
        print("=" * 70)
        downloaded_files = {}
        
        for key, file_info in found_files.items():
            filepath = self.download_file(file_info['url'], file_info['filename'])
            if filepath:
                downloaded_files[key] = filepath
            time.sleep(0.5)  # Be polite to the server
        
        if not downloaded_files:
            print("✗ No files were downloaded. Exiting.\n")
            return
        
        print(f"✓ Successfully downloaded {len(downloaded_files)} files\n")
        
        # Step 5: Process files
        print("[Step 5/5] Processing downloaded files...")
        
        # Process "All Forms" file
        if 'all_forms' in downloaded_files:
            print("\n" + "=" * 70)
            print("PROCESSING: ALL FORMS FILE")
            print("=" * 70)
            
            all_forms_path = downloaded_files['all_forms']
            
            # Extract rows 16-17
            self.extract_rows_16_17(all_forms_path)
            
            # Analyze and extract I-129 and I-140 data
            self.analyze_all_forms(all_forms_path)
        
        # Process I-140 specific files
        self.process_i140_files(downloaded_files)
        
        # Final summary
        print("=" * 70)
        print("EXTRACTION COMPLETE!")
        print("=" * 70)
        print(f"\nFiscal Year: {self.fiscal_year}, Quarter: {self.quarter}")
        print(f"Output location: {self.output_dir.absolute()}")
        print(f"\nFiles created:")
        print(f"  📁 downloads/ ({len(downloaded_files)} Excel files)")
        print(f"  📄 rows_16_17_I129_I140.csv")
        print(f"  📄 I129_data.csv")
        print(f"  📄 I140_data.csv")
        print(f"  📄 I129_I140_summary.json")
        print(f"  📄 Processed I-140 CSV reports")
        print()

def extract_multiple_quarters(quarters_list, output_dir='./uscis_output'):
    """
    Extract data for multiple quarters
    
    Args:
        quarters_list: List of tuples [(fiscal_year, quarter), ...]
                      Example: [(2025, 3), (2025, 2), (2024, 4)]
        output_dir: Base output directory
    """
    print("=" * 70)
    print("MULTI-QUARTER EXTRACTION")
    print("=" * 70)
    print(f"Extracting data for {len(quarters_list)} quarters\n")
    
    results = []
    
    for fy, q in quarters_list:
        print(f"\n{'='*70}")
        print(f"Starting extraction for FY{fy} Q{q}")
        print(f"{'='*70}\n")
        
        extractor = USCISDataExtractor(
            fiscal_year=fy,
            quarter=q,
            output_dir=output_dir
        )
        
        try:
            extractor.run_extraction()
            results.append({
                'fiscal_year': fy,
                'quarter': q,
                'status': 'success',
                'output_dir': str(extractor.output_dir)
            })
        except Exception as e:
            print(f"✗ Error extracting FY{fy} Q{q}: {e}\n")
            results.append({
                'fiscal_year': fy,
                'quarter': q,
                'status': 'failed',
                'error': str(e)
            })
        
        time.sleep(2)  # Wait between quarters
    
    # Save summary
    summary_path = Path(output_dir) / 'extraction_summary.json'
    with open(summary_path, 'w') as f:
        json.dump(results, f, indent=2)
    
    print("\n" + "=" * 70)
    print("MULTI-QUARTER EXTRACTION COMPLETE")
    print("=" * 70)
    print(f"Summary saved to: {summary_path}")
    print(f"\nResults:")
    for r in results:
        status_icon = "✓" if r['status'] == 'success' else "✗"
        print(f"  {status_icon} FY{r['fiscal_year']} Q{r['quarter']}: {r['status']}")
    print()

if __name__ == "__main__":
    import sys
    
    if len(sys.argv) >= 3:
        # Command line: python script.py <fiscal_year> <quarter> [output_dir]
        fy = int(sys.argv[1])
        q = int(sys.argv[2])
        output = sys.argv[3] if len(sys.argv) > 3 else './uscis_output'
        
        extractor = USCISDataExtractor(fiscal_year=fy, quarter=q, output_dir=output)
        extractor.run_extraction()
    
    elif len(sys.argv) == 2 and sys.argv[1] == '--multi':
        # Multi-quarter mode
        # Extract last 4 quarters as example
        current_fy = 2025
        quarters = [
            (2025, 3),
            (2025, 2),
            (2025, 1),
            (2024, 4),
        ]
        extract_multiple_quarters(quarters)
    
    else:
        # Default: FY2025 Q3
        extractor = USCISDataExtractor(fiscal_year=2025, quarter=3)
        extractor.run_extraction()

USCIS H1B DATA EXTRACTOR
Target: Fiscal Year 2025, Quarter 3

[Step 1/5] Fetching USCIS data index page...
Fetching: https://www.uscis.gov/tools/reports-and-studies/immigration-and-citizenship-data
✓ Status: 200

[Step 2/5] Finding FY2025 Q3 page...
Searching for FY2025 Q3 page...
  → Found candidate: All USCIS Application and Petition Form Types (Fiscal Year 2025, Quart...
  → Found candidate: Form I-765, Application for Employment Authorization, Eligibility Cate...
  → Found candidate: Form I-765, Application for Employment Authorization Counts of Pending...
  → Found candidate: Form I-526, Immigrant Petition by Standalone Investor and Form I-526E,...
  → Found candidate: Form I-130, (Awaiting Visa Availability) (Fiscal Year 2025, Quarter 3)...
  → Found candidate: Form I-360, Petition for Special Immigrant with a Classification of Sp...
  → Found candidate: Form I-140, I-360, I-526 Approved EB Petitions Awaiting Visa Final Pri...
  → Found candidate: Form I-907, Request for Premium 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


SEARCHING FOR TARGET FILES
⚠ Missing files: i140_quarter, i140_preference, all_forms, eb_awaiting
These files may not be available for this quarter yet.

✗ No target files found. Exiting.

